TIL about Ibis[1] which is a data analysis framework designed with the goal of making Python a first class citizen of Hadoop Ecosystem. I am excited about this project for couple of reasons.

1) The main person behind the project is Wes Mckinney [2] who is the guy who built Pandas and revolutionized the PyData Ecosystem

2) Ibis is going to be built on top of Impala[3] which is a distributed SQL engine which used HDFS for storage and is written in C++ . Impala is one of the best choices we have currently when it comes to "SQL on Hadoop". 

3) This blogpost [4] is mind-blowing where the examples show how Ibis converts the Python code to spaghetti complex SQL that nobody would want to write but is blazingly fast. Ibis provides a rich DSL that would help us to write clean and maintainable Python code but without going through the ordeal of writing lots of SQL code.

I think this project has a great future ahead.

[1] http://www.ibis-project.org/
[2] http://wesmckinney.com/
[3] http://impala.io/
[4] http://blog.ibis-project.org/sqlite-crunchbase-quickstart/

In [16]:
#pip install ibis
import ibis
ibis.options.interactive = True

In [4]:
# connecting to the table

con = ibis.sqlite.connect('crunchbase.db')

In [6]:
#Listing all the tables

con.list_tables()

[u'acquisitions', u'companies', u'investments', u'rounds']

In [10]:
#Creating a pointer to companies table

company_tb = con.table('companies')


In [11]:
#Schema 

company_tb

SQLiteTable[table]
  name: companies
  schema:
    permalink : string
    name : string
    homepage_url : string
    category_list : string
    market : string
    funding_total_usd : double
    status : string
    country_code : string
    state_code : string
    region : string
    city : string
    funding_rounds : int64
    founded_at : string
    founded_month : string
    founded_quarter : string
    founded_year : double
    first_funding_at : string
    last_funding_at : string

In [13]:
#Getting some quick stats

company_tb.info()

Table rows: 54292

Column             Type    Non-null #
------             ----    ----------
permalink          string  51166     
name               string  51166     
homepage_url       string  47583     
category_list      string  47826     
market             string  46416     
funding_total_usd  double  42237     
status             string  50081     
country_code       string  46294     
state_code         string  31603     
region             string  46095     
city               string  45460     
funding_rounds     int64   51166     
founded_at         string  39833     
founded_month      string  39755     
founded_quarter    string  39755     
founded_year       double  39755     
first_funding_at   string  51166     
last_funding_at    string  51166     

In [18]:
# Select * from companies limit 5

company_tb.limit(5)

                         permalink                name  \
0              /organization/-fame               #fame   
1            /organization/hashoff            #HASHOFF   
2            /organization/waywire            #waywire   
3  /organization/tv-communications  &TV Communications   
4    /organization/rock-your-paper   'Rock' Your Paper   

                   homepage_url                               category_list  \
0            http://livfame.com                                     |Media|   
1        http://www.hashoff.com       |Digital Media|Internet|Social Media|   
2        http://www.waywire.com  |Entertainment|Politics|Social Media|News|   
3         http://enjoyandtv.com                                     |Games|   
4  http://www.rockyourpaper.org                      |Publishing|Education|   

            market  funding_total_usd     status country_code state_code  \
0           Media            10000000  operating          IND       None   
1   Digital Media       

In [19]:
magic = company_tb.limit(5).execute()

In [20]:
magic

,permalink,name,homepage_url,category_list,market,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,founded_month,founded_quarter,founded_year,first_funding_at,last_funding_at
0,/organization/-fame,#fame,http://livfame.com,|Media|,Media,10000000,operating,IND,None,Mumbai,Mumbai,1,None,None,None,NaN,2015-01-05,2015-01-05
1,/organization/hashoff,#HASHOFF,http://www.hashoff.com,|Digital Media|Internet|Social Media|,Digital Media,455000,operating,USA,CO,Denver,Denver,1,2014-04-01 00:00:00,2014-04,2014-Q2,2014,2014-12-08,2014-12-08
2,/organization/waywire,#waywire,http://www.waywire.com,|Entertainment|Politics|Social Media|News|,News,1750000,acquired,USA,NY,New York City,New York,1,2012-06-01 00:00:00,2012-06,2012-Q2,2012,2012-06-30,2012-06-30
3,/organization/tv-communications,&TV Communications,http://enjoyandtv.com,|Games|,Games,4000000,operating,USA,CA,Los Angeles,Los Angeles,2,None,None,None,NaN,2010-06-04,2010-09-23
4,/organization/rock-your-paper,'Rock' Your Paper,http://www.rockyourpaper.org,|Publishing|Education|,Publishing,40000,operating,EST,None,Tallinn,Tallinn,1,2012-10-26 00:00:00,2012-10,2012-Q4,2012,2012-08-09,2012-08-09


In [22]:
type(magic)

pandas.core.frame.DataFrame

In [23]:
#Max

company_tb.funding_total_usd.max()

30079503000.0

In [25]:
#100 Million Folks

bigshots = company_tb.filter(company_tb.funding_total_usd>100000000)

In [26]:
bigshots.count()

1086

In [29]:
bigshots.filter(bigshots.country_code=='IND').count()


22

In [33]:
indian_bigshots = bigshots.filter(bigshots.country_code=='IND')
indian_bigshots.execute()

,permalink,name,homepage_url,category_list,market,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,founded_month,founded_quarter,founded_year,first_funding_at,last_funding_at
0,/organization/avantha,Avantha,http://avanthagroup.com,|Hardware + Software|,Hardware + Software,150000000,operating,IND,None,New Delhi,New Delhi,1,None,None,None,NaN,2013-12-16,2013-12-16
1,/organization/capturesolar-energy,CaptureSolar Energy,http://www.capturesolar.com/cms/,|Solar|Environmental Innovation|Renewable Ener...,Energy,125000000,operating,IND,None,Pune,Pune,1,2013-01-01 00:00:00,2013-01,2013-Q1,2013,2014-06-01,2014-06-01
2,/organization/flipkart,Flipkart,http://www.flipkart.com,|Online Shopping|E-Commerce|,Online Shopping,2451140000,operating,IND,None,Bangalore,Bangalore,11,2007-09-01 00:00:00,2007-09,2007-Q3,2007,2009-01-01,2014-12-21
3,/organization/gland-pharma,Gland Pharma,None,|Biotechnology|,Biotechnology,200000000,operating,IND,None,Hyderabad,Hyderabad,1,None,None,None,NaN,2013-11-28,2013-11-28
4,/organization/gmr-group,GMR Group,http://www.gmrgroup.in,|Consulting|,Consulting,398350000,operating,IND,None,Bangalore,Bangalore,1,1978-01-01 00:00:00,1978-01,1978-Q1,1978,2014-01-27,2014-01-27
5,/organization/greenko-group,Greenko Group,http://www.greenkogroup.com,|Clean Technology|,Clean Technology,241983286,operating,IND,None,Hyderabad,Hyderabad,2,None,None,None,NaN,2010-01-28,2014-10-24
6,/organization/haier,Haier,http://haier.com/in,|Home Automation|,Home Automation,364000000,operating,IND,None,New Delhi,New Delhi,1,None,None,None,NaN,2013-12-08,2013-12-08
7,/organization/housing-com,Housing.com,https://housing.com,|Real Estate|,Real Estate,139500000,operating,IND,None,Mumbai,Mumbai,4,2012-06-07 00:00:00,2012-06,2012-Q2,2012,2013-06-04,2014-11-19
8,/organization/jasper,Jasper,http://www.snapdeal.com,|E-Commerce|,E-Commerce,102000000,None,IND,None,New Delhi,New Delhi,3,None,None,None,NaN,2011-01-25,2014-04-28
9,/organization/kalyan-jewellers,Kalyan Jewellers,http://www.kalyanjewellers.net/,|Jewelry|,Jewelry,200000000,operating,IND,None,IND - Other,None,1,1993-01-01 00:00:00,1993-01,1993-Q1,1993,2014-08-26,2014-08-26


In [53]:
#Sorting is ascending by default . Need to pass False as keyword in a *tuple*

expr = indian_bigshots['name','funding_total_usd','funding_rounds']
sorted_expr = expr.sort_by((expr.funding_total_usd,False))
sorted_expr.execute()

,name,funding_total_usd,funding_rounds
0,Flipkart,2451140000,11
1,Reliance Jio Infocomm Ltd.,2250000000,2
2,Snapdeal,1062700000,8
3,GMR Group,398350000,1
4,Haier,364000000,1
5,Tower Vision,300000000,1
6,Ola (ANI Technologies Pvt. Ltd.),276830000,5
7,ReNew Power,275000000,2
8,Greenko Group,241983286,2
9,Gland Pharma,200000000,1


In [54]:
print ibis.sqlite.compile(sorted_expr)

SELECT t0.name, t0.funding_total_usd, t0.funding_rounds 
FROM (SELECT t1.name AS name, t1.funding_total_usd AS funding_total_usd, t1.funding_rounds AS funding_rounds 
FROM companies AS t1 
WHERE t1.funding_total_usd > ? AND t1.country_code = ?) AS t0 ORDER BY t0.funding_total_usd DESC


In [55]:
sorted_expr.schema()

ibis.Schema {  
  name               string
  funding_total_usd  double
  funding_rounds     int64
}